In [0]:
%pip install torch torchvision torchaudio transformers faiss-cpu

  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/41/33/0f21b15f8e231bb55578f6b32e8c18675585b7bf97cb0aee96b1591e4193/torchaudio-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/51/b2/4f9abd2b859cef0e2332d3ff032e1973281fac1204fa8da14effc326f528/faiss_cpu-1.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/17/ed/d6e73554726a4aea054353093f3dffe68ea18f006d14c0604e651f3e6ee0/torchaudio-2.5.0-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/cc/f3/a950329a25ee1af14c05065ce6c1751f031de9e6d5eebb0620ce3d0938ed/to

In [0]:
%pip install numpy

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
import numpy as np
import faiss

# Step 1: Embed Documents (same as before)
def embed_documents(documents):
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    embeddings = []
    with torch.no_grad():
        for doc in documents:
            inputs = tokenizer(doc, return_tensors='pt', padding=True, truncation=True)
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).numpy()
            embeddings.append(embedding)
    return np.vstack(embeddings)

# Step 2: Create FAISS Vector Index (same as before)
def create_vector_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

# Step 3: Search Similar Documents (same as before)
def search_similar_documents(index, query_embedding, top_k=5):
    distances, indices = index.search(query_embedding, top_k)
    return distances, indices

# Step 4: Retrieve Relevant Context (same as before)
def retrieve_context(query, index, documents):
    query_embedding = embed_documents([query])
    distances, indices = search_similar_documents(index, query_embedding)
    return [documents[i] for i in indices[0]]

# Step 5: Generate Response Using Retrieved Context
def generate_response(query, context_docs):
    model_name = "gpt2"  # or any other causal language model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # Combine the query and retrieved documents as context for generation
    context_text = " ".join(context_docs)
    prompt = f"Query: {query}\nContext: {context_text}\nAnswer:"
    
    # Tokenize input and generate response
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=150, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example Documents
documents = [
    "AI is transforming healthcare.",
    "Machine learning is a subset of artificial intelligence.",
    "Deep learning enables many modern AI applications.",
    "Natural Language Processing is a field of AI.",
    "AI technologies are becoming more pervasive in various industries."
]

# Embedding and Indexing
document_embeddings = embed_documents(documents)
index = create_vector_index(document_embeddings)



tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [0]:
 # Query and Retrieve Context
query = "How is AI used in healthcare?"
retrieved_context = retrieve_context(query, index, documents)

# Generate Response
generated_response = generate_response(query, retrieved_context)

# Output Retrieved Context and Generated Response
print("Retrieved Context for Query:")
for doc in retrieved_context:
    print(f"Document: '{doc}'")

print("\nGenerated Response:")
print(generated_response)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/databricks/python/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Retrieved Context for Query:
Document: 'AI is transforming healthcare.'
Document: 'AI technologies are becoming more pervasive in various industries.'
Document: 'Machine learning is a subset of artificial intelligence.'
Document: 'Natural Language Processing is a field of AI.'
Document: 'Deep learning enables many modern AI applications.'

Generated Response:
Query: How is AI used in healthcare?
Context: AI is transforming healthcare. AI technologies are becoming more pervasive in various industries. Machine learning is a subset of artificial intelligence. Natural Language Processing is a field of AI. Deep learning enables many modern AI applications.
Answer: AI is transforming healthcare. AI technologies are becoming more pervasive in various industries. Machine learning is a subset of artificial intelligence. Natural Language Processing is a field of AI. Deep learning enables many modern AI applications.
Question: How is AI used in healthcare?
Context: AI is transforming healthcare. 